# Case Study Unit 4 - Signal Frontier Analysis
### Cory Adams, Chris Boomhower, Alexandra Fisher, Alex Frye
### *MSDS 7333, September 23, 2017*

Assignments:
* Abstract - Chris
* Intro - Chris
* Methods - Chris
* Results 1,2 - Frye
* Results 3-6 Overview - Fisher
* Results 3,4 - Fisher
* Results 5,6 - Cory
* Conclusion/Future Works - Chris

## Abstract
While the history of financial trade markets as we know them today spans centuries, the adoption of Python to study market trends has really only gained traction since 2005 [XXX XXX]. The language has been used to study everything from historical stock price trends to rolling correlation and linear regression. This case study is performed to demonstrate Python's effectiveness for studying stock cumulative returns and to test the principles of short, intermediate, and long term momentum. The dataset analyzed consists of a mock-up portfolio of five semiconductor industry stocks. Prices were pulled over a ten year timeframe and cumulative returns explored. Momentums were calculated for various lookback and holding periods with the expectation that short and long term periods would exhibit inverted outcomes and intermediate term periods would exhibit a continued trend over future periods of equivalent length. Finally, research's theory for best strategy being a lookback period of one year with a holding period of three months is tested using this portfolio data [XXX - <span style="color:red">use reference from Youtube video</span>]. Next steps for using the R statistical language to replicate this analysis are finally discussed.

## Introduction
Financial stock market data can be rather challenging to analyze without adequate tools. Fortunately, libraries such as Python's *pandas* module make working with financial and time-series data very straightforward. The *pandas* library efficiently indexes time-sensitive data, supports operations for time series of different frequencies, handles "as of" data imputation, and even provides functions conducive to stock value analysis with respect to time.

To evaluate *pandas* capabilities, we put the toolset to work on momentum analysis. Existing research indicates three types of momentum: 1) Short (1 month), 2) Intermediate (6-12 months), and 3) Long (3-5 years) [XXX - <span style="color:red">use references shared in Youtube video</span>]. This research reveals that both short and long term momentums exhibit reversal in short and long term future values. In contrast, intermediate term momentum demonstrates continued stock portfolio performance over intermediate future periods. Additionally, studies argue the best strategy is to use a lookback period of one year with a holding period of three months when determining when to buy or sell [XXX - <span style="color:red">use references shared in Youtube video</span>]. These are interesting principles and their analysis using real, historical stock values make for a great case study to evaluate Python and *pandas* for financial data applications.

In [ ]:
## Load necessary modules
import pandas_datareader as web
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from collections import defaultdict
import datetime
from IPython.display import display
import missingno as msno

%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Methods
Methods here
1. Pull 10 years worth of data, leaving enough buffer for lookback
2. Exploratory - review cumulative returns (percent change day-to-day). Review 10 years in full but then also again for most recent quarter as well.
3. Analyze Short
4. Analyze Intermediate
5. Analyze Long term momentum (Be sure to tie back to existing research since we are basically just testing the existing theories for momentum).
6. Review results against theory that 12 month loopback with 3 month holding is best strategy




* Describe momentum computation for simplified porfolio
* Texas Instruments, ON Semiconductor, Nexperia, Diodes, and Maxim stocks identified for this case study

* Note that code adopted from *Python for Data Analysis*, Chapter 11, pages 345-347




## Results
Results here

* Number each step as outlined in Methods section
* If some expounding is needed, feel free to add code chunks

### 1) Loading the Data

To begin our analysis, the first step is to load data for our target stocks. Utilizing the "pandas_datareader" python package, we are able to capture adjusted close data from yahoo finance. In order to account for momentum calculations, including prior year statistics, we have decided to pull a full 10 years of historical business day prices for this analysis from 6-30-2007 to 6-30-2017 as may be seen in rendered header and tail output below. 

In [ ]:
## Note that if RemoteDataError received upon chunk run, rerun this chunk until data pulled successfully
names = ['TXN', 'ON', 'NXPI', 'DIOD', 'MXIM']
def get_px(stock, start, end):
    return web.DataReader(stock, 'yahoo', start, end)

s = datetime.datetime(2007, 6, 30)
e = datetime.datetime(2017, 6, 29) #Offset by one day

px = ''
tryCnt = 1
while(tryCnt >0):
    try:
        px = pd.DataFrame({n:get_px(n, s, e)['Adj Close'] for n in names})
        print("Successful Data Load on attempt {0}.".format(tryCnt))
        tryCnt = -1
    except:
        print("Error Occurred on attempt",tryCnt, "... Trying again.")
        tryCnt+=1
display(px.head())
display(px.tail())

**Missing Data** Upon our initial loading of the data we see that there are missing values for NXPI. To gain high level exposure to missing values within this dataset, we have produced a missing value plot below. In this plot, we identify that the only stock which we do not have the full 10 year date span, is NXPI. This is expected, as can be confirmed through NASDAQ**[XXX]**, NXPI began on the market on 8/6/2010. All data prior to this time for NXPI, has resulted in NaN values. 

In [ ]:
    #Plot for missing Data
filtered_msnoData = msno.nullity_sort(msno.nullity_filter(px))
msno.matrix(filtered_msnoData)

del filtered_msnoData

Shown below, we may confirm that this data has loaded properly, and missing values stop after 2010-08-05. This matches our findings on NASDAQ, and we may proceed with confidence that data has loaded successfully.

In [ ]:
display(pd.DataFrame(pd.concat([pd.Series(px[px["NXPI"].isnull()].NXPI.tail()), pd.Series(px[px["NXPI"].notnull()].NXPI.head())])).T)

### 2) Exploratory Data Analysis - Stock Price and Cumulative Returns

To explore these stocks, prior to further analysis, we have plotted both stock prices and cumulative returns across the most recent 3 Month, 1 Year, 3 Year, and 10 Year Periods. These plots are divided into 3 rows: 1) Stock Prices; 2) Cumulative Returns with Standardized Scale; 3) Cumulative Returns with Zoomed Scale. To provide a better picture of the data, cumulative returns are displayed with two different y scale ranges.

**Stock Prices** While we do not have the NXPI data element prior to August 2010, it is definitely apparent that NXPI has had a tremendously successful time in the market since inception. With prices ranging from approximately \$14 in 2010 all the way to approximately $110 in 2017, it is followed next by TXN with an ending price of only approximately \$80 in 2017. Although NXPI has had the most successful upward trend in the long-term, it also appears the most volatile. We notice, especially in the 3 year range, that there are much larger peaks and valleys than in the other prices. This factor likely attributes NXPI as a higher risk stock option than others plotted at its side. 

**Cumulative Returns** The formula for cumulative returns is:

<center> $\frac{(Current Price of Security - Original Price of Security)}{(Original Price of Security)}$**[XXX]** </center>

This computation allows us to assess aggregate gain or loss from an initialized starting date. These plots provided display return percentages in decimal form. The second row of cumulative return plots provide us context toward aggregate gains in the short term versus long term, showing the powerful plunge of NXPI in the early years of inception. However, what was difficult to understand in the stock price visualizations, is that NXPI does not actually take the win in the intermediate 1-3 year terms. As we dive into the scaled visuals of these same values we find that DIOD, second to last in cumulative returns over the 10 year period, takes the win by nearly 20\% in the 1 year period. This goes to show, that buy and hold methodologies are very important to your overall investment strategy!

In [ ]:
plt.rcParams['figure.figsize'] = (18, 9)
fig = plt.figure()

###### Stock Prices ######
# 3 month
ax = fig.add_subplot(3,4,1)
ax.set_ylabel("Stock Price")
px.ix['2017-03-30':'2017-06-29'].plot(ax=ax, title = "Stock Prices, 3 Month Lookback")

# 1 Year
ax = fig.add_subplot(3,4,2)
ax.set_ylabel("Stock Price")
px.ix['2016-06-30':'2017-06-29'].plot(ax=ax, title = "Stock Prices, 1 Year Lookback")

# 3 Year
ax = fig.add_subplot(3,4,3)
ax.set_ylabel("Stock Price")
px.ix['2014-06-30':'2017-06-29'].plot(ax=ax, title = "Stock Prices, 3 Year Lookback")

# 10 Year
ax = fig.add_subplot(3,4,4)
ax.set_ylabel("Stock Price")
px.plot(ax=ax, title = "Stock Prices, 10 Year Lookback")

###### Cum Return ######
# 3 month
ax = fig.add_subplot(3,4,5)
ax.set_ylabel("Cum. Returns")
px_pad = px.ix['2017-03-30':'2017-06-29'].asfreq('B').fillna(method='ffill') #Keep at business day freq and forward fill any missing values
rets = px_pad.pct_change() #Plot day-to-day percent change in value
((1 + rets).cumprod() - 1).plot(ax=ax, title = "Cumulative Returns, 3 Month Lookback", ylim=(-1,7),rot=45)

# 1 Year
ax = fig.add_subplot(3,4,6)
ax.set_ylabel("Cum. Returns")
px_pad = px.ix['2016-06-30':'2017-06-29'].asfreq('B').fillna(method='ffill') #Keep at business day freq and forward fill any missing values
rets = px_pad.pct_change() #Plot day-to-day percent change in value
((1 + rets).cumprod() - 1).plot(ax=ax, title = "Cumulative Returns, 1 Year Lookback", ylim=(-1,7),rot=45)

# 3 Year
ax = fig.add_subplot(3,4,7)
ax.set_ylabel("Cum. Returns")
px_pad = px.ix['2014-06-30':'2017-06-29'].asfreq('B').fillna(method='ffill') #Keep at business day freq and forward fill any missing values
rets = px_pad.pct_change() #Plot day-to-day percent change in value
((1 + rets).cumprod() - 1).plot(ax=ax, title = "Cumulative Returns, 3 Year Lookback", ylim=(-1,7),rot=45)

# 10 Year
ax = fig.add_subplot(3,4,8)
ax.set_ylabel("Cum. Returns")
px_pad = px.asfreq('B').fillna(method='ffill') #Keep at business day freq and forward fill any missing values
rets = px_pad.pct_change() #Plot day-to-day percent change in value
((1 + rets).cumprod() - 1).plot(ax=ax, title = "Cumulative Returns, 10 Year Lookback", ylim=(-1,7),rot=45)


###### Cum Return ZOOM ######
# 3 month
ax = fig.add_subplot(3,4,9)
ax.set_ylabel("Cum. Returns")
px_pad = px.ix['2017-03-30':'2017-06-29'].asfreq('B').fillna(method='ffill') #Keep at business day freq and forward fill any missing values
rets = px_pad.pct_change() #Plot day-to-day percent change in value
((1 + rets).cumprod() - 1).plot(ax=ax, title = "Cum. Returns, 3 Month Lookback-Scaled",rot=45)

# 1 Year
ax = fig.add_subplot(3,4,10)
ax.set_ylabel("Cum. Returns")
px_pad = px.ix['2016-06-30':'2017-06-29'].asfreq('B').fillna(method='ffill') #Keep at business day freq and forward fill any missing values
rets = px_pad.pct_change() #Plot day-to-day percent change in value
((1 + rets).cumprod() - 1).plot(ax=ax, title = "Cum. Returns, 1 Year Lookback-Scaled",rot=45)

# 3 Year
ax = fig.add_subplot(3,4,11)
ax.set_ylabel("Cum. Returns")
px_pad = px.ix['2014-06-30':'2017-06-29'].asfreq('B').fillna(method='ffill') #Keep at business day freq and forward fill any missing values
rets = px_pad.pct_change() #Plot day-to-day percent change in value
((1 + rets).cumprod() - 1).plot(ax=ax, title = "Cum. Returns, 3 Year Lookback-Scaled",rot=45)

# 10 Year
ax = fig.add_subplot(3,4,12)
ax.set_ylabel("Cum. Returns")
px_pad = px.asfreq('B').fillna(method='ffill') #Keep at business day freq and forward fill any missing values
rets = px_pad.pct_change() #Plot day-to-day percent change in value
((1 + rets).cumprod() - 1).plot(ax=ax, title = "Cum. Returns, 10 Year Lookback-Scaled",rot=45)

fig.subplots_adjust(top = 0.99, bottom=0.01, hspace=.5, wspace=0.2)

### 3-6 Overview

In [ ]:
def calc_mom(price, lookback, lag):
    mom_ret = price.shift(lag).pct_change(lookback)
    ranks = mom_ret.rank(axis=1, ascending=False)
    demeaned = ranks.subtract(ranks.mean(axis=1), axis=0)
    return demeaned.divide(demeaned.std(axis=1), axis=0)

In [ ]:
compound = lambda x : (1 + x).prod() - 1
daily_sr = lambda x: x.mean() / x.std()

def strat_sr(prices, lb, hold):
    # Compute portfolio weights
    freq = '%dB' % hold
    port = calc_mom(prices, lb, lag=1)

    daily_rets = prices.pct_change()

    # Compute portfolio returns
    port = port.shift(1).resample(freq).first()
    returns = daily_rets.resample(freq).apply(compound)
    port_rets = (port * returns).sum(axis=1)

    return daily_sr(port_rets) * np.sqrt(252 / hold)

Compute Sharpe Ratio values for various combinations of lookback and holding periods.

* **Analyze Short, Intermediate, and Long term momentum (as described in youtube video referenced in References section.**
* Existing research indicates best strategy is lookback of 12 months and holding period of 3 months

In [ ]:
%%time
def sr_combos(startLB, endLB, stepLB, startHD, endHD, stepHD):
    lookbacks = range(startLB, endLB, stepLB)
    holdings = range(startHD, endHD, stepHD)
    dd = defaultdict(dict)
    for lb in lookbacks:
        for hold in holdings:
            dd[lb][hold] = strat_sr(px, lb, hold)

    ddf = pd.DataFrame(dd)
    ddf.index.name = 'Holding Period'
    ddf.columns.name = 'Lookback Period'
    
    return ddf

In [ ]:
def heatmap(df, normalize, cmap=plt.cm.bwr):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    axim = ax.imshow(df.values, cmap=cmap, interpolation='nearest', norm=normalize)
    ax.set_xlabel(df.columns.name)
    ax.set_xticks(np.arange(len(df.columns)))
    ax.set_xticklabels(list(df.columns))
    ax.set_ylabel(df.index.name)
    ax.set_yticks(np.arange(len(df.index)))
    ax.set_yticklabels(list(df.index))
    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(9) 
        tick.label.set_rotation('vertical')
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(9)
    plt.colorbar(axim)

### 3

In [ ]:
%time short = sr_combos(1, 31, 1, 1, 62, 2)

In [ ]:
normalize = mpl.colors.Normalize(vmin=-5, vmax=5)
heatmap(short, normalize)

### 4

<div class="alert alert-block alert-info">

Fisher, be sure to leave majority of 'best strategy' discussions to Cory in step 6 (he will be discussing 12 month lookback and 3 month holding there in greater detail).

In [ ]:
%time intermediate = sr_combos(1, 365, 14, 1, 730, 30)

In [ ]:
normalize = mpl.colors.Normalize(vmin=-5, vmax=5)
heatmap(intermediate, normalize)

### 5

In [ ]:
%time long = sr_combos(1, 1095, 60, 1, 2190, 120)

In [ ]:
max(long.max(axis=1))

In [ ]:
#normalize = mpl.colors.Normalize(vmin=min(long.min(axis=1)), vmax=-min(long.min(axis=1)))
normalize = mpl.colors.Normalize(vmin=-5, vmax=5)
heatmap(long, normalize)

### 6

Portfolio example Sharpe Ratio for 1 year lookback and 3 month holding.

<div class="alert alert-block alert-info">

Cory, feel free to expound in terms of code as needed. Much of this section will likely be discussing the intermediate momentum heatmap for 3 month holdings, but if you think it would be helpful to visualize anything or output any other specific values, please do so.

In [ ]:
strat_sr(px, 365, 90)

## Discussion and Future Work
Discussion, conclusions and future work here

## References
W. McKinney, Python for Data Analysis. Beijing: OReilly, 2014.

I. Staff, “Momentum,” Investopedia, 29-Jul-2015. [Online]. Available: http://www.investopedia.com/terms/m/momentum.asp. [Accessed: 23-Sep-2017].

YouTube, 11-Sep-2017. [Online]. Available: https://www.youtube.com/watch?v=KQJYb5YFIoE. [Accessed: 24-Sep-2017].

[1] Enter<br>

J. Hur, “History of The Stock Market,” BeBusinessEd, 05-Sep-2017. [Online]. Available: https://bebusinessed.com/history/history-of-the-stock-market/. [Accessed: 25-Sep-2017].